<a href="https://colab.research.google.com/github/compartia/AI-tecture/blob/master/miro_openai_pipe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade openai typing-extensions

In [ ]:
import openai
from google.colab import userdata
# openai.api_key =

ai_client = openai.OpenAI(api_key=userdata.get('OP_API')  )

In [ ]:
BOARD_ID = 'uXjVN61x8Pg='

# Miro client

In [ ]:
import requests
import pandas as pd


class MiroClient:
  def __init__(self, token, board_id):
    self.board_id = board_id
    self.token = token
    self.base_url = f"https://api.miro.com/v2/boards/{board_id}"

    self.headers = {
        "accept": "application/json",
        "authorization": f"Bearer {self.token}"
    }


  def get_items(self, cursor=None, limit=50):
    limit_p = f'limit={limit}'

    url = f"{self.base_url}/items?{limit_p}"
    if cursor is not None:
      url = f"{self.base_url}/items?cursor={cursor}&{limit_p}"

    response = requests.get(url, headers=self.headers)
    return response

  def get_connectors(self, cursor=None, limit=50):
    limit_p = f'limit={limit}'

    url = f"{self.base_url}/connectors?{limit_p}"
    if cursor is not None:
      url = f"{self.base_url}/connectors?cursor={cursor}&{limit_p}"

    response = requests.get(url, headers=self.headers)
    return response


  def get_all_pages(self, page_method):
    # print('get_all_pages, page_method=', page_method)
    cursor = None

    while True:
      response = page_method(cursor)
      rj = response.json()
      yield rj
      cursor = rj.get('cursor')
      if cursor is None:
        #TODO: safety! add limit : we trust 3rd pary apy too much, what if cursor is never None?
        break


miro_client = MiroClient(userdata.get('MIRO_TOOKEN'), BOARD_ID)


In [ ]:
r = miro_client.get_items(limit=50).json()

for k in r['data']:
  print()
  print('-'*30)
  print(k)

In [ ]:
type(r['data'])

## Getting all connectors

In [ ]:
def decode_connectors(page:dict):
  # print(page)
  for i in page['data']:
    yield i['id'], i.get('startItem', {}).get('id'), i.get('endItem', {}).get('id')

## test it
one_page_con = miro_client.get_connectors(limit=10).json()
x = [i for i in decode_connectors(one_page_con)]
x

In [ ]:
def get_all_connectors(self):
  for page in self.get_all_pages(self.get_connectors):
    # print('get_all_connectors, page:', page)
    for i in decode_connectors(page):
      yield i


MiroClient.get_all_connectors = get_all_connectors


##-----
miro_client = MiroClient(userdata.get('MIRO_TOOKEN'), BOARD_ID)
all_connectors_iter = miro_client.get_all_connectors()

connectors_df = pd.DataFrame(all_connectors_iter, columns=['id','id_from', 'id_to'])
connectors_df = connectors_df.set_index('id')
connectors_df


## Getting all shapes (widgets)

In [ ]:
def decode_items(page:dict):

  for i in page['data']:
    # print(type(i))
    if type(i) == dict :
      yield i['id'], i['type'], i['data'].get('shape', '_undefined_'), i['data'].get('content'), i['modifiedAt'], i['geometry'], i['position']

    else:
      print('NOT A DICT!!', type(i), i)
      decode_items(i)

In [ ]:


def get_all_items(self):
  for page in self.get_all_pages(self.get_items):
    # print( page.get('cursor'), page.get('size'), page.get('total') )
    for i in decode_items(page):
      yield i
      # print(i)


MiroClient.get_all_items=get_all_items

miro_client = MiroClient(userdata.get('MIRO_TOOKEN'), BOARD_ID)
items_iter = miro_client.get_all_items()

df = pd.DataFrame(items_iter, columns=['id','type', 'shape', "contents", 'modifiedAt', 'geometry', 'position'])
df = df.set_index('id')

In [ ]:
shapes_df =df
shapes_df

### Get shapes with prompts

In [ ]:
prompts_df = shapes_df[shapes_df['shape']=='parallelogram']
prompts_df

# Test open AI

### Find connectors of prompts-shapes (connectors of prallelograms)

In [ ]:
#incoming shapes
for id in prompts_df.index:
  print()
  print('-'*40)
  print('prompt', id, prompts_df.loc[id]['contents'])

  incoming_links = connectors_df[ connectors_df['id_to'] ==id]
  # print (links ['id_from'])
  for _, l in incoming_links.iterrows():
    print(l)
    print()
    incoming_shape_id = l['id_from']
    print('SHAPE  FROM:', incoming_shape_id)
    if incoming_shape_id in shapes_df.index:

      print('text:', id, shapes_df.loc[incoming_shape_id]['contents'])
    else:
      print('ERROR!', incoming_shape_id, 'not found!' )

  #   print(l)

In [ ]:
items = miro_client.get_items()
items

# -----





# Google Ngrams api test



In [ ]:
import requests
import json
import re
import pandas as pd

def get_ngram_data(word, corpus='ru-2019', start_year=2000, end_year=2020, smoothing=3):
    """
    Fetches ngram data from Google Books Ngram Viewer.

    :param word: Word to search for.
    :param corpus: Corpus ID (default is 15 for English).
    :param start_year: Start year for the data.
    :param end_year: End year for the data.
    :param smoothing: Smoothing parameter.
    :return: A list of tuples with year and corresponding frequency.
    """
    url = f"https://books.google.com/ngrams/json?content={word}&year_start={start_year}&year_end={end_year}&corpus={corpus}&smoothing={smoothing}"

    response = requests.get(url)
    if response.status_code != 200:
        return "Failed to retrieve data"

    data = json.loads(response.text)
    print (data)

    # Parse the data
    if data:
        ngram_data = []
        for entry in data[0]['timeseries']:
            year = start_year
            ngram_data.append((year, entry))
            start_year += 1
        return ngram_data
    else:
        return "No data found"

# Example usage
word = "нечто"
ngram_data = get_ngram_data(word)
print(ngram_data)


In [ ]:
class Observer:
  def __init__(self):
    self.state = [] #memory
    self.observed = [] #sensory

    print('inited', self.state)

  def observe(self, state):
    self.observed = state
    print('observed', self.observed)
    self.compare()

  def compare(self):
    if self.state
    pass


s = [1]

o = Observer()
o.observe(s)

In [ ]:
498501 - 497503

In [ ]:
import pandas as pd

# Convert to DataFrame
if isinstance(ngram_data, list):
    df = pd.DataFrame(ngram_data, columns=['Year', 'Frequency'])
    print(df['Frequency'].mean())
else:
    print(ngram_data)

In [ ]:
!pip install umap-learn


In [ ]:
# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import umap

# 1) Constructing a random matrix of distances
def create_random_distance_matrix(size):
    matrix = np.random.rand(size, size)
    matrix = (matrix + matrix.T) / 2  # Making the matrix symmetric
    np.fill_diagonal(matrix, 0)  # Setting diagonal to zero (distance to itself)
    return matrix

# 2) Using t-SNE to fit these distances into 2D and plotting the result
def plot_with_tsne(matrix):
    tsne = TSNE(n_components=2, metric="precomputed")
    embedded_2d = tsne.fit_transform(matrix)

    # Plotting
    plt.scatter(embedded_2d[:, 0], embedded_2d[:, 1])
    plt.title("2D representation of nodes using t-SNE")
    plt.xlabel("Dimension 1")
    plt.ylabel("Dimension 2")
    plt.show()

def plot_with_umap(matrix):
    reducer = umap.UMAP(metric="precomputed")
    embedded_2d = reducer.fit_transform(matrix)

    # Plotting
    plt.scatter(embedded_2d[:, 0], embedded_2d[:, 1])
    plt.title("2D representation of nodes using UMAP")
    plt.xlabel("Dimension 1")
    plt.ylabel("Dimension 2")
    plt.show()

# Example usage
size = 10  # Number of nodes
distance_matrix = create_random_distance_matrix(size)
plot_with_umap(distance_matrix)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Function to create a random square matrix representing a graph
def create_random_graph(size):
    matrix = np.random.rand(size, size)
    np.fill_diagonal(matrix, 0)  # setting diagonal to zero to eliminate self loops
    return matrix

# Function to plot the matrix
def plot_matrix(matrix, title="Graph Matrix"):
    plt.figure(figsize=(4, 4))
    plt.imshow(matrix, cmap='viridis')
    plt.colorbar()
    plt.title(title)
    plt.show()

# Greedy Traveling Salesman Algorithm
def greedy_tsp(matrix):
    n = len(matrix)
    visited = [False] * n
    path = []
    current = 0
    visited[current] = True
    path.append(current)

    while len(path) < n:
        next_city = np.argmin([matrix[current, j] if not visited[j] else np.inf for j in range(n)])
        path.append(next_city)
        visited[next_city] = True
        current = next_city

    path.append(path[0]) # Returning to the starting point
    return path

# Function to plot the path
def plot_path(matrix, path):
    plt.figure(figsize=(4, 4))
    plt.imshow(matrix, cmap='viridis')
    for i in range(len(path) - 1):
        y, x = path[i], path[i+1]
        plt.plot([x, y], [x, y], 'ro-')
    plt.colorbar()
    plt.title("Path on Graph")
    plt.show()

# Create a random graph
size = 4  # You can change the size of the matrix
matrix = create_random_graph(size)

# Plot the matrix
plot_matrix(matrix)

# Find and plot the path
path = greedy_tsp(matrix)
plot_path(matrix, path)


In [ ]:
import itertools

def calculate_total_distance(path, matrix):
    """Calculate the total distance of a given path using the distance matrix."""
    total_distance = 0
    for i in range(len(path) - 1):
        total_distance += matrix[path[i], path[i + 1]]
    total_distance += matrix[path[-1], path[0]]  # Return to the starting point
    return total_distance

def exact_tsp(matrix):
    n = len(matrix)
    cities = list(range(n))
    shortest_path = None
    min_distance = np.inf

    for path in itertools.permutations(cities[1:]):  # All permutations of cities, excluding the first city
        path = (0,) + path  # Adding the starting city to the beginning
        total_distance = calculate_total_distance(path, matrix)

        if total_distance < min_distance:
            min_distance = total_distance
            shortest_path = path

    return shortest_path, min_distance

# Find the exact solution for the TSP
exact_path, exact_total_cost = exact_tsp(matrix)
print(f"Exact Path: {exact_path}, Total Cost: {exact_total_cost}")
plot_path(matrix, exact_path)

In [ ]:
# Function to plot the path
def plot_path(matrix, path):
    plt.figure(figsize=(4, 4))
    plt.imshow(matrix, cmap='viridis')
    for i in range(len(path) - 1):
        y, x = path[i], path[i+1]
        plt.plot([x, y], [x, y], 'ro-', alpha=0.2)
    plt.colorbar()
    plt.title("Path on Graph")
    plt.show()

plot_path(matrix, exact_path)